In [1]:
import numpy as np
import xarray as xr
import dask
import pandas as pd
import os
import sys
import cftime
from xarray.coding.times import decode_cf_datetime

# 1. Sea-ice input files

In [2]:
seaice_hadisst = xr.open_dataset('/g/data/q90/ac9768/ancil/inputs/HadISST_ice.nc')
seaice_input4mips = xr.open_dataset('/g/data/q90/ac9768/ancil/inputs/siconc_input4MIPs_SSTsAndSeaIce_CMIP_PCMDI-AMIP-1-1-8_gn_187001-202112.nc')

In [14]:
seaice_hadisst_062024_122024 = seaice_hadisst.sel(time=slice('2024-06-01','2024-12-31'))
add_year = seaice_hadisst_062024_122024['time'] + np.timedelta64(365, 'D')
seaice_hadisst_062025_122025 = seaice_hadisst_062024_122024.assign_coords(time=add_year)
seaice_hadisst_extended = xr.concat([seaice_hadisst,seaice_hadisst_062025_122025],dim='time')
seaice_hadisst_extended

<xarray.Dataset> Size: 485MB
Dimensions:    (time: 1872, nv: 2, latitude: 180, longitude: 360)
Coordinates:
  * time       (time) datetime64[ns] 15kB 1870-01-16T11:59:59.505615234 ... 2...
  * latitude   (latitude) float32 720B 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
  * longitude  (longitude) float32 1kB -179.5 -178.5 -177.5 ... 178.5 179.5
Dimensions without coordinates: nv
Data variables:
    time_bnds  (time, nv) float32 15kB 0.0 31.0 31.0 ... 5.658e+04 5.661e+04
    sic        (time, latitude, longitude) float32 485MB 1.0 1.0 1.0 ... nan nan
Attributes:
    Title:                      Monthly version of HadISST sea ice component
    description:                HadISST sea ice concentration.
    institution:                Met Office Hadley Centre
    source:                     HadISST
    reference:                  Rayner, N. A., Parker, D. E., Horton, E. B., ...
    Conventions:                CF-1.0
    history:                    2/7/2025 converted to netcdf from pp format
    supplementary_information:  Updates and supplementary information will be...
    comment:                    Data restrictions: for academic research use ...

In [18]:
seaice_hadisst_extended['time'] = seaice_hadisst_extended.time.dt.round('h')
seaice_hadisst_rename_vars = seaice_hadisst_extended.sortby('latitude').rename({'latitude': 'lat', 'longitude': 'lon','nv':'bnds','sic':'siconc'})
seaice_hadisst_rename_vars['lon'] = (seaice_hadisst_rename_vars['lon']) % 360
seaice_hadisst_rename_vars = seaice_hadisst_rename_vars.sortby('lon')
seaice_hadisst_rename_vars['siconc'] = seaice_hadisst_rename_vars['siconc']*100
seaice_hadisst_time_slice_end2024 = seaice_hadisst_rename_vars.sel(time=slice('1870-01-01', '2025-12-31'))
seaice_hadisst_time_slice_end2024['siconc']=seaice_hadisst_time_slice_end2024['siconc'].where(~np.isnan(seaice_hadisst_time_slice_end2024['siconc']),0)

# make attrs for hadisst identical to input4mips
seaice_hadisst_time_slice_end2024.lon.attrs = seaice_input4mips.lon.attrs
seaice_hadisst_time_slice_end2024.lat.attrs = seaice_input4mips.lat.attrs
seaice_hadisst_time_slice_end2024.time.attrs = seaice_input4mips.time.attrs
seaice_hadisst_time_slice_end2024.siconc.attrs = seaice_input4mips.siconc.attrs
seaice_hadisst_time_slice_end2024['lat_bnds'] = seaice_input4mips['lat_bnds']
seaice_hadisst_time_slice_end2024['lon_bnds'] = seaice_input4mips['lon_bnds']

# change time_bnds from float32 to datetime64
# Set known units and calendar manually
units = 'days since 1870-01-01'
calendar = 'gregorian'
# Decode the float time bounds
decoded_bnds = decode_cf_datetime(
    seaice_hadisst_time_slice_end2024['time_bnds'],
    units=units,
    calendar=calendar
)
# Replace time_bnds in the dataset with decoded values
seaice_hadisst_time_slice_end2024['time_bnds'] = (seaice_hadisst_time_slice_end2024['time_bnds'].dims,decoded_bnds)
seaice_hadisst_time_slice_end2024['time_bnds'] = seaice_hadisst_time_slice_end2024.time_bnds.dt.round('h')

# seaice_hadisst_time_slice_end2024.to_netcdf("/g/data/q90/ac9768/ancil/inputs/hadisst_seaice_end2025.nc")

In [19]:
seaice_hadisst_time_slice_end2024

<xarray.Dataset> Size: 485MB
Dimensions:    (time: 1872, bnds: 2, lat: 180, lon: 360)
Coordinates:
  * time       (time) datetime64[ns] 15kB 1870-01-16T12:00:00 ... 2025-12-16T...
  * lat        (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 30kB 1870-01-01 ... 2025-01-01
    siconc     (time, lat, lon) float32 485MB 0.0 0.0 0.0 ... 100.0 100.0 100.0
    lat_bnds   (lat, bnds) float64 3kB ...
    lon_bnds   (lon, bnds) float64 6kB ...
Attributes:
    Title:                      Monthly version of HadISST sea ice component
    description:                HadISST sea ice concentration.
    institution:                Met Office Hadley Centre
    source:                     HadISST
    reference:                  Rayner, N. A., Parker, D. E., Horton, E. B., ...
    Conventions:                CF-1.0
    history:                    2/7/2025 converted to netcdf from pp format
    supplementary_information:  Updates and supplementary information will be...
    comment:                    Data restrictions: for academic research use ...

# 2. SST input files

In [21]:
sst_hadisst = xr.open_dataset('/g/data/q90/ac9768/ancil/inputs/HadISST_sst.nc')
sst_input4mips = xr.open_dataset('/g/data/q90/ac9768/ancil/inputs/input4mips_tos_1870-2021-12.nc')

In [26]:
sst_hadisst_062024_122024 = sst_hadisst.sel(time=slice('2024-06-01','2024-12-31'))
add_year = sst_hadisst_062024_122024['time'] + np.timedelta64(365, 'D')
sst_hadisst_062025_122025 = sst_hadisst_062024_122024.assign_coords(time=add_year)
sst_hadisst_extended = xr.concat([sst_hadisst,sst_hadisst_062025_122025],dim='time')

<xarray.DataArray 'time' ()> Size: 8B
array('2025-05-16T12:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 8B 2025-05-16T12:00:00
Attributes:
    long_name:      Time
    standard_name:  time

In [27]:
sst_hadisst_extended['time'] = sst_hadisst_extended.time.dt.round('h')
sst_hadisst_rename_vars = sst_hadisst_extended.sortby('latitude').rename({'latitude': 'lat', 'longitude': 'lon','nv':'bnds','sst':'tos'})
sst_hadisst_rename_vars['tos'].attrs['units'] = 'degC'
sst_hadisst_rename_vars['lon'] = (sst_hadisst_rename_vars['lon']) % 360 
sst_hadisst_rename_vars = sst_hadisst_rename_vars.sortby('lon')
sst_hadisst_rename_vars.lon.attrs = sst_hadisst_rename_vars.lon.attrs
sst_hadisst_time_slice_end2024 = sst_hadisst_rename_vars.sel(time=slice('1870-01-01', '2025-12-31'))
sst_hadisst_time_slice_end2024['tos']=sst_hadisst_time_slice_end2024['tos'].where(sst_hadisst_time_slice_end2024.tos>-1.8,np.isnan)

# make attrs for hadisst identical to input4mips
sst_hadisst_time_slice_end2024.lon.attrs = sst_input4mips.lon.attrs
sst_hadisst_time_slice_end2024.lat.attrs = sst_input4mips.lat.attrs
sst_hadisst_time_slice_end2024.time.attrs = sst_input4mips.time.attrs
sst_hadisst_time_slice_end2024.tos.attrs = sst_input4mips.tos.attrs
sst_hadisst_time_slice_end2024['lat_bnds'] = sst_input4mips['lat_bnds']
sst_hadisst_time_slice_end2024['lon_bnds'] = sst_input4mips['lon_bnds']

# change time_bnds from float32 to datetime64
# Set known units and calendar manually
units = 'days since 1870-01-01'
calendar = 'gregorian'
# Decode the float time bounds
decoded_bnds = decode_cf_datetime(
    sst_hadisst_time_slice_end2024['time_bnds'],
    units=units,
    calendar=calendar
)
# Replace time_bnds in the dataset with decoded values
sst_hadisst_time_slice_end2024['time_bnds'] = (sst_hadisst_time_slice_end2024['time_bnds'].dims,decoded_bnds)
sst_hadisst_time_slice_end2024['time_bnds'] = sst_hadisst_time_slice_end2024.time_bnds.dt.round('h')

sst_hadisst_time_slice_end2024.to_netcdf("/g/data/q90/ac9768/ancil/inputs/hadisst_sst_end2025.nc")

In [28]:
sst_hadisst_time_slice_end2024

<xarray.Dataset> Size: 485MB
Dimensions:    (time: 1872, bnds: 2, lat: 180, lon: 360)
Coordinates:
  * time       (time) datetime64[ns] 15kB 1870-01-16T12:00:00 ... 2025-12-16T...
  * lat        (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon        (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 30kB 1870-01-01 ... 2025-01-01
    tos        (time, lat, lon) float32 485MB 1.0 1.0 1.0 1.0 ... 0.0 0.0 0.0
    lat_bnds   (lat, bnds) float64 3kB ...
    lon_bnds   (lon, bnds) float64 6kB ...
Attributes:
    Title:                      Monthly version of HadISST sea surface temper...
    description:                HadISST 1.1 monthly average sea surface tempe...
    institution:                Met Office Hadley Centre
    source:                     HadISST
    reference:                  Rayner, N. A., Parker, D. E., Horton, E. B., ...
    Conventions:                CF-1.0
    history:                    2/7/2025 converted to netcdf from pp format
    supplementary_information:  Updates and supplementary information will be...
    comment:                    Data restrictions: for academic research use ...